In [1]:
import pandas as pd
import numpy as np
import random
import copy
import statsmodels.discrete.discrete_model as sm

In [2]:
PATH = "/Users/Amy/Dropbox/OSPC - Shared/MEPS/"

In [3]:
raw_MEPS = pd.read_csv(str(PATH + 'h171.csv'))

/Users/Amy/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1322,1403,1436,1522,1605,1620,1621,1627,1636,1687,1737,1744,1747,1748,1756,1759,1765,1772,1775,1776,1779,1782,1793,1807) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
id_for_analysis = ['DUPERSID','PANEL', 'WAGEP14X', 'REGION14', 'PERWT14F', 'SEX','AGE14X', 'TOTMCR14']
MEPS_medicare = raw_MEPS[id_for_analysis]

In [5]:
MEPS_medicare['yes_to_mc'] = np.where(MEPS_medicare.TOTMCR14!=0, 1, 0)

/Users/Amy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [6]:
MEPS_medicare = MEPS_medicare[MEPS_medicare.yes_to_mc==1]

In [7]:
MEPS_medicare.SEX = np.where(MEPS_medicare.SEX=='2 FEMALE', 'Female', 'Male')

In [8]:
MEPS_medicare.REGION14 = MEPS_medicare.REGION14.str.split(' ', expand=True, n = 1).get(1).values
MEPS_medicare.REGION14 = MEPS_medicare.REGION14.str.capitalize()

In [9]:
CPS = pd.read_csv('/Users/Amy/Dropbox/asec2013_pubuse.csv')

/Users/Amy/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5,23,24,29,83,265,282) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
medicare_columns = ['mcare','peridnum','marsupwt', 'wsal_val', 'a_age', 'a_sex', 'gereg', 'h_seq',
                    'ss_yn', 'ss_val']
CPS = CPS[medicare_columns]

In [11]:
CPS.mcare = np.where(CPS.mcare=='Yes', 1, 0)

In [12]:
CPS.ss_val = np.where(CPS.ss_val=='None or not in universe', 0, CPS.ss_val)
CPS.ss_val = pd.to_numeric(CPS.ss_val)
CPS.ss_yn = np.where(CPS.ss_yn=='Yes', 1, 0)

In [13]:
CPS.wsal_val = np.where(CPS.wsal_val=="None or not in universe", 0, CPS.wsal_val)
CPS.wsal_val = pd.to_numeric(CPS.wsal_val)

In [14]:
CPS.a_age = np.where(CPS.a_age == "80-84 years of age",
                     random.randrange(80, 84),
                     CPS.a_age)
CPS.a_age = np.where(CPS.a_age == "85+ years of age",
                     random.randrange(85, 95),
                     CPS.a_age)
CPS.a_age = pd.to_numeric(CPS.a_age)

In [15]:
CPS['MEPS_ID'] = np.zeros(len(CPS))

In [16]:
CPS['age_65'] = np.where(CPS.a_age>=65, 1, 0)

In [17]:
model = sm.Logit(endog=CPS.mcare, exog=CPS[['ss_yn', 'wsal_val','a_age','age_65']])
results = model.fit()
print results.summary()

Optimization terminated successfully.
         Current function value: 0.193541
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                  mcare   No. Observations:               202634
Model:                          Logit   Df Residuals:                   202630
Method:                           MLE   Df Model:                            3
Date:                Fri, 04 Aug 2017   Pseudo R-squ.:                  0.5046
Time:                        09:51:17   Log-Likelihood:                -39218.
converged:                       True   LL-Null:                       -79164.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
ss_yn          7.6522      0.055    138.708      0.000         7.544     7.760
wsal_val   -1.724e-05   7.85

In [18]:
CPS_population = copy.deepcopy(CPS)
CPS_population['prob'] = results.predict()

In [19]:
CPS_population[['peridnum', 'prob']].to_csv('medicare_prob.csv', index=False)

In [20]:
CPS = CPS[CPS.mcare==1]

In [21]:
len(CPS)

26802

In [22]:
Region = ['Northeast', 'South', 'Midwest', 'West']
Gender = ['Male', 'Female']

In [23]:
random.seed(1)

In [24]:
for this_area in Region:
    for this_gender in Gender:
        subset_CPS = CPS[(CPS.gereg==this_area)*(CPS.a_sex==this_gender)]
        MEPS_donor = MEPS_medicare[(MEPS_medicare.REGION14==this_area)*(MEPS_medicare.SEX==this_gender)]
        
        for i, record in subset_CPS.iterrows():
            age_range = [record.a_age - 2, record.a_age + 2]
            income_range = [record.wsal_val - 100, record.wsal_val + 100]
            
            f1 = (MEPS_donor.AGE14X >= age_range[0])
            f2 = (MEPS_donor.AGE14X <= age_range[1])
            f3 = (MEPS_donor.WAGEP14X >= income_range[0])
            f4 = (MEPS_donor.WAGEP14X <= income_range[1])
            pool = MEPS_donor[f1 * f2 * f3 * f4]
            
            number_donors = len(pool)
            if number_donors < 1:
                # release income constraints and find the person w/ closest income
                pool = MEPS_donor[f1 * f2]
                number_donors = len(pool)
                
                if number_donors < 1:
                    if record.a_age < 85:
                        print 'dont have anyone in this age range'
                        print age_range
                        continue
                    else:
                        pool = MEPS_donor[MEPS_donor.AGE14X==85]

                closest_wage = min(pool.WAGEP14X, key=lambda x:abs(x-record.wsal_val))
                CPS.MEPS_ID[CPS.peridnum==record.peridnum] = pool.DUPERSID[pool.WAGEP14X==closest_wage].values[0]    

            else:
                row_number = random.randint(1, number_donors) - 1
                index = pool.DUPERSID.index[row_number]
                CPS.MEPS_ID[CPS.peridnum==record.peridnum] = pool.DUPERSID.loc[index] 

/Users/Amy/anaconda/lib/python2.7/site-packages/pandas/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))
/Users/Amy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


dont have anyone in this age range
[10, 14]
dont have anyone in this age range
[10, 14]
dont have anyone in this age range
[-2, 2]
dont have anyone in this age range
[-1, 3]
dont have anyone in this age range
[10, 14]
dont have anyone in this age range
[10, 14]
dont have anyone in this age range
[-2, 2]
dont have anyone in this age range
[10, 14]
dont have anyone in this age range
[-1, 3]
dont have anyone in this age range
[-1, 3]
dont have anyone in this age range
[6, 10]
dont have anyone in this age range
[24, 28]
dont have anyone in this age range
[6, 10]
dont have anyone in this age range
[12, 16]
dont have anyone in this age range
[12, 16]
dont have anyone in this age range
[12, 16]
dont have anyone in this age range
[6, 10]
dont have anyone in this age range
[24, 28]
dont have anyone in this age range
[24, 28]
dont have anyone in this age range
[24, 28]
dont have anyone in this age range
[24, 28]
dont have anyone in this age range
[24, 28]
dont have anyone in this age range
[24, 

/Users/Amy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [25]:
CPS.marsupwt[CPS.MEPS_ID!=0].sum()

48833689.480000146

In [26]:
CPS['DUPERSID'] = CPS.MEPS_ID
CPS = pd.merge(CPS, MEPS_medicare, on='DUPERSID', how='left')

In [27]:
(CPS.marsupwt*CPS.TOTMCR14)[CPS.MEPS_ID!=0].sum()/1000000000

393.0893501562817

In [28]:
ratio = 576.0000/((CPS.marsupwt*CPS.TOTMCR14)[CPS.MEPS_ID!=0].sum()/1000000000)

In [29]:
CPS["MedicareX"] = np.zeros(len(CPS))
CPS.MedicareX = CPS.TOTMCR14 * ratio

In [30]:
CPS[['peridnum', 'DUPERSID', 'MedicareX', 'h_seq']].to_csv('medicare12.csv', index=False)